<a href="https://colab.research.google.com/github/jolinayhsueh/Energy-/blob/main/Supply_forecasting_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MP model




This model is to forecast minerals supply by each country's production and reserves. Time series forecasting method is used for projecting ___ years or supply.

### Data Collection

### Analysis

---
### Coding

Packages:
*   Data preperation: `Numpy`
*   Forecasting
*   Machine learning
*   Data Visualization

Model Sections:
*  [Data preparation](https://colab.research.google.com/drive/1G4WS7w_iM4Ai4h0SlZOCPSuHgArYnYNa#scrollTo=3fY3S6KGaUd4&line=1&uniqifier=1)
*   Model (blah blah)

Data source:
*   [USGS](https://www.sciencebase.gov/catalog/item/63b5f411d34e92aad3caa57f)
*   [World mining data](https://www.world-mining-data.info/?World_Mining_Data___Data_Section)

# Data preparation

In [30]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.getcwd()
os.chdir('/content/drive/MyDrive/MP Model')
os.getcwd()
import pandas as pd #imports panda as data frame
import numpy as np #imports numpy as data frame

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
# Install packages
!pip install pystan fbprophet
!pip install pre pycaret

  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  Using cached cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached setuptools_git-1.2-py2.py3-none-any.whl (10 kB)
  Using cached PyMeeus-0.5.12-py3-none-any.whl
  Using cached ephem-4.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.0 kB)
Using cached ephem-4.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet
  Running setup.py clean for fbprophet
Failed to build fbprophet
ERROR: Could not build wheels for fbprophet, which is required to install pyproject.

In [43]:
# Importing required libraries
import numpy as np
import pandas as pd, datetime
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from time import time
import os
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.tsa.arima_model import  ARIMA
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score
from pandas import DataFrame
import xgboost as xgb
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')

In [33]:
#Read data

supply_df = pd.read_excel(io="/content/drive/MyDrive/MP Model/Supply Reserves.xlsx")

#Reserves and Prod data
reserves = pd.read_excel(io="/content/drive/MyDrive/MP Model/Supply Reserves.xlsx",sheet_name="Reserves",keep_default_na=False)
prod = pd.read_excel(io="/content/drive/MyDrive/MP Model/Supply Reserves.xlsx",sheet_name="Production",keep_default_na=False)
proddf = pd.read_excel(io="/content/drive/MyDrive/MP Model/Supply Reserves.xlsx",sheet_name="Production",keep_default_na=False)

In [34]:
reserves.head()
proddf.head()

,Country,Mineral,year,Prod_m.t
0,Argentina,Li,2017,12360
1,Australia,Li,2017,98650
2,Bolivia,Li,2017,20
3,Brazil,Li,2017,533
4,Canada,Li,2017,66


In [39]:
# Convert data fromat

proddf['year'] = pd.to_datetime(proddf['year']).dt.strftime('%Y')
proddf.dropna(inplace=True)
proddf['Country'] = proddf['Country'].astype(str)
proddf['Mineral'] = proddf['Mineral'].astype(str)

proddf.head()

,Country,Mineral,year,Prod_m.t
0,Argentina,Li,1970,12360
1,Australia,Li,1970,98650
2,Bolivia,Li,1970,20
3,Brazil,Li,1970,533
4,Canada,Li,1970,66


In [40]:
# proddf = proddf.reset_index()
proddf=proddf.rename(columns={'year':'ds', 'Prod_m.t':'y'})
proddf.head()

,Country,Mineral,ds,y
0,Argentina,Li,1970,12360
1,Australia,Li,1970,98650
2,Bolivia,Li,1970,20
3,Brazil,Li,1970,533
4,Canada,Li,1970,66


In [48]:
#Create forecast library
forecasts = {}

for country in proddf['Country'].unique():
    for mineral in proddf['Mineral'].unique():
        country_mineral_df = proddf[(proddf['Country'] == country) & (proddf['Mineral'] == mineral)]
        if not country_mineral_df.empty:
            # Prophet requires the columns ds (date) and y (value)
            ts_df = country_mineral_df.rename(columns={'ds': 'ds', 'y': 'y'})

            # Create and fit the model
            model = Prophet()
            model.fit(ts_df)

            # Create future dataframe and forecast
            future = model.make_future_dataframe(periods=29, freq='Y')
            forecast = model.predict(future)

            # Store forecast data
            forecasts[(country, mineral)] = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfnlfwq0e/o2a9g2hm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfnlfwq0e/omwuxe74.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=92503', 'data', 'file=/tmp/tmpfnlfwq0e/o2a9g2hm.json', 'init=/tmp/tmpfnlfwq0e/omwuxe74.json', 'output', 'file=/tmp/tmpfnlfwq0e/prophet_modelh5_dkkql/prophet_model-20240123153613.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
15:36:13 - cmdstanpy - INFO - Chain [1] start processing
IN

RuntimeError: Error during optimization! Command '/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin random seed=92503 data file=/tmp/tmpfnlfwq0e/o2a9g2hm.json init=/tmp/tmpfnlfwq0e/omwuxe74.json output file=/tmp/tmpfnlfwq0e/prophet_modelh5_dkkql/prophet_model-20240123153613.csv method=optimize algorithm=newton iter=10000' failed: console log output:

method = optimize
  optimize
    algorithm = newton
      newton
    jacobian = 0 (Default)
    iter = 10000
    save_iterations = 0 (Default)
id = 1 (Default)
data
  file = /tmp/tmpfnlfwq0e/o2a9g2hm.json
init = /tmp/tmpfnlfwq0e/omwuxe74.json
random
  seed = 92503
output
  file = /tmp/tmpfnlfwq0e/prophet_modelh5_dkkql/prophet_model-20240123153613.csv
  diagnostic_file =  (Default)
  refresh = 100 (Default)
  sig_figs = -1 (Default)
  profile_file = profile.csv (Default)
num_threads = 1 (Default)

Rejecting initial value:
  Error evaluating the log probability at the initial value.
Exception: normal_lpdf: Random v

In [ ]:
# Create a DataFrame
prod_fc_df = pd.DataFrame(forecasts)

# Export to Excel
file_name = 'production forecast_data.xlsx'
prod_fc_df.to_excel(file_name, index=False)

print(f"Data exported to {file_name}")